In [1]:
import numpy as np
import pyvista as pv
from kale.algorithms import contour_banded
%config InlineBackend.figure_format = "retina"


In [2]:
# Requires PyVista 0.38.1
pv.set_jupyter_backend("server")
pv.global_theme.trame.interactive_ratio = 2
pv.global_theme.trame.still_ratio = 2
pv.global_theme.image_scale = 2  # upscales the saved screenshots/video frames

In [3]:
MESH_GEOMETRY_FILE_NAME = "nankai_121728_clean.vtk"
MESH_VALUES_FILE_NAME = "2023_03_04_00_02_03.hdf"
MESH_Z_SCALE = 0.025
TIME_STEP_INTERESTING = 1000
CONTOUR_LINE_WIDTH = 1
CONTOUR_LINE_COLOR = "black"
COLOR_MAPS = {
    "cumulative_slip": "CET_L19",  #'CET_C3',
    "geometric_moment": "CET_R3",
    "last_event_slip": "CET_L19",
    "loading_rate": "CET_C3",
}

COLORBAR_FONT_SIZE = 34
AXES_FONT_SIZE = 26
TIMESTEP_FONT_SIZE = 12

SCALAR_BAR_OPTS = dict(
    # Position (coordinates are on the viewport)
    vertical=True,
    height=0.20,
    width=0.05,
    position_x=0.05,
    position_y=0.40,
    # Labels
    title_font_size=AXES_FONT_SIZE,
    label_font_size=AXES_FONT_SIZE,
    n_labels=3,
    italic=False,
    fmt="%.1f",
    font_family="arial",
    shadow=True,
)

# Load Data

In [7]:
import xarray as xr
MESH_GEOMETRY_FILE_NAME = "nankai_121728_clean.vtk"
MESH_VALUES_FILE_NAME = "2023_03_04_00_02_03.hdf"
ds = xr.open_dataset(MESH_VALUES_FILE_NAME, engine="netcdf4")
# data = ds["loading_rate"][0, :]
data = ds["cumulative_slip"][40000, :]
mesh = pv.read(MESH_GEOMETRY_FILE_NAME)
mesh.points[:, -1] *= MESH_Z_SCALE

# Extract boundary of mesh
boundary = mesh.extract_feature_edges(
    boundary_edges=True,
    non_manifold_edges=False,
    feature_edges=False,
    manifold_edges=False,
)

# # Load fields from .npz file
# npz_data = np.load("skies_single_step_erosion_figures_nankai.npz")
# mesh.cell_data["pre_event_slip_deficit"] = npz_data["name1"]
# mesh.cell_data["event_slip"] = npz_data["name2"]
# mesh.cell_data["post_event_slip_deficit"] = npz_data["name3"]
# mesh.cell_data["meshes_areas"] = npz_data["name4"]

SCALAR_BAR_OPTS = dict(
    vertical=True,
    height=0.50,
    width=0.10,
    position_x=0.05,
    position_y=0.40,
    title_font_size=10,
    label_font_size=10,
    n_labels=3,
    italic=False,
    fmt="%.1f",
    font_family="arial",
    shadow=False,
)





# Direct linear mapping

In [8]:
mesh.cell_data["data"] = ds["cumulative_slip"][40000, :]

pl = pv.Plotter()
pl.add_mesh(boundary, color=CONTOUR_LINE_COLOR, line_width=CONTOUR_LINE_WIDTH)
box = pv.Box(mesh.bounds)
pl.add_mesh(box, color="lightgrey", culling="front")
CONTOUR_LEVELS = np.linspace(0, 20, 10)
N_COLORS = len(CONTOUR_LEVELS) - 1
CLIM = [np.min(CONTOUR_LEVELS), np.max(CONTOUR_LEVELS)]

contour, edges = contour_banded(
    mesh,
    CONTOUR_LEVELS,
    rng=CLIM,
    scalars="data",
)
pl.add_mesh(
    contour,
    cmap="RdYlBu_r",
    clim=CLIM,
    scalars="data",
    n_colors=N_COLORS,
    scalar_bar_args=dict(title=f"", **SCALAR_BAR_OPTS),
)
pl.add_mesh(
    edges.GetOutputPort(1), color=CONTOUR_LINE_COLOR, line_width=CONTOUR_LINE_WIDTH
)
pl.show()


Widget(value="<iframe src='http://localhost:51612/index.html?ui=P_0x16aeba310_1&reconnect=auto' style='width: …

# Plot of same data but with log scaling
# I don't know how to get the black contour lines

In [22]:
mesh.cell_data["data"] = np.log10(ds["cumulative_slip"][40000, :])

pl = pv.Plotter()
pl.add_mesh(boundary, color=CONTOUR_LINE_COLOR, line_width=CONTOUR_LINE_WIDTH)
box = pv.Box(mesh.bounds)
pl.add_mesh(box, color="lightgrey", culling="front")
CONTOUR_LEVELS = np.linspace(0, 20, 10)
CONTOUR_LEVELS = np.logspace(-1, 0, 10)
N_COLORS = len(CONTOUR_LEVELS) - 1
CLIM = [np.min(CONTOUR_LEVELS), np.max(CONTOUR_LEVELS)]

contour, edges = contour_banded(
    mesh,
    CONTOUR_LEVELS,
    rng=CLIM,
    scalars="data",
)
pl.add_mesh(
    contour,
    cmap="RdYlBu_r",
    clim=CLIM,
    scalars="data",
    n_colors=N_COLORS,
    scalar_bar_args=dict(title=f"", **SCALAR_BAR_OPTS),
)
pl.add_mesh(
    edges.GetOutputPort(1), color=CONTOUR_LINE_COLOR, line_width=CONTOUR_LINE_WIDTH
)
pl.show()


/Users/meade/mambaforge/envs/kale/lib/python3.11/site-packages/xarray/core/computation.py:760: RuntimeWarning: divide by zero encountered in log10
  result_data = func(*input_data)
/Users/meade/mambaforge/envs/kale/lib/python3.11/site-packages/xarray/core/computation.py:760: RuntimeWarning: invalid value encountered in log10
  result_data = func(*input_data)


Widget(value="<iframe src='http://localhost:51612/index.html?ui=P_0x16aeebe90_11&reconnect=auto' style='width:…